In [1]:
import numpy as np
import pandas as pd
import xarray as xr

import os
from glob import glob

import matplotlib.pyplot as plt

import histlib.matchup as match
import histlib.diagnosis as diag
from histlib.cstes import labels, zarr_dir, matchup_dir, var
from histlib.matchup import _data_var, _stress_var, _aviso_var

from xhistogram.xarray import histogram


import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.geodesic as cgeo
crs = ccrs.PlateCarree()
import cmocean.cm as cm


/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


In [2]:
l = "gps_Sentinel-3_A_2019"#labels[0]

In [3]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    #cluster = PBSCluster(cores=56, processes=28, walltime='04:00:00')
    #cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    cluster = PBSCluster(cores=28, processes=28, walltime='08:00:00')
    w = cluster.scale(jobs=1)
else:
    from dask.distributed import Client, LocalCluster
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.64:8787/status,
Dashboard: http://10.148.1.64:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.64:37038,Workers: 0
Dashboard: http://10.148.1.64:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## IMPORT DATA, build dataset
We choose to get only erastar wind term, and we reject PEACHY data, and we select only deltaT<1H

In [4]:
dl=5# degree
DL =25e3 #meters
DT = 0.5*3600 #seconds

#IF restricted computation
wd_x= ['es_cstrio_z15_alti_wd_x', 'es_cstrio_z15_drifter_wd_x']
wd_y = []
grad_x = ['alti_ggx_adt_filtered',
          'alti_ggx_adt_filtered_ocean_tide',
          'alti_ggx_adt_filtered_ocean_tide_internal_tide',
          'alti_ggx_adt_filtered_ocean_tide_internal_tide_dac',
         'aviso_alti_ggx_adt',
          'aviso_drifter_ggx_adt'
         ]
grad_y =[]
cutoff=[0, 2.5]


var = wd_x+grad_x+grad_y+['drifter_acc_x_0','drifter_coriolis_x_0']


def bin_lonlat_ms(ds,l, dl):
    dsm = match.add_except_sum(ds, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff)  
    dsm = dsm.reset_coords(['lon', 'lat', 'time']).drop(['id_comb', 'time'])
    dfm = dsm.to_dask_dataframe().set_index('obs')
    dfm["latbin"] = (dfm.lat // dl) * dl
    dfm["lonbin"] = (dfm.lon // dl) * dl
    #ms
    d2 = dfm.drop(['lon', 'lat', 'lonbin', 'latbin'], axis=1)**2
    dd = dfm[['lonbin', 'latbin']].merge(d2)
    dd = dd.groupby(["latbin", "lonbin"]).mean()
    
    #count
    dnb = dfm.reset_index()[['obs', 'latbin', 'lonbin']].groupby(["latbin", "lonbin"]).count().obs.compute().to_xarray()
    dsms = dd.compute().to_xarray()
    #attrs
    for v in list(dsms.variables) :
        if v in ds :
            dsms[v].attrs = ds[v].attrs
    #merge
    dso = xr.merge([dsms, dnb.rename('nb_coloc_bin')])
    dso['drifter_sat_year']=l
    dso = dso.expand_dims('drifter_sat_year')
    dso = dso.set_coords('drifter_sat_year')
    # center lon, lat bins + reindex to have same for all
    lon_bins, lat_bins = np.arange(-180, 180, dl), np.arange(-90, 90, dl)
    dso = dso.reindex({'lonbin':lon_bins, 'latbin':lat_bins})
    dso['lonbin'] = dso['lonbin']+dl/2
    dso['latbin'] = dso['latbin']+dl/2

    return dso

def bin_lonlat_mean(ds,l, dl):
    dsm = match.add_except_sum(ds, wd_x=wd_x, wd_y=wd_y, grad_x=grad_x, grad_y=grad_y, cutoff=cutoff)    
    dsm = dsm.reset_coords(['lon', 'lat', 'time']).drop(['id_comb', 'time'])
    dfm = dsm.to_dask_dataframe().set_index('obs')
    dfm["latbin"] = (dfm.lat // dl) * dl
    dfm["lonbin"] = (dfm.lon // dl) * dl
    #ms
    d2 = dfm.drop(['lon', 'lat', 'lonbin', 'latbin'], axis=1)
    dd = dfm[['lonbin', 'latbin']].merge(d2)
    dd = dd.groupby(["latbin", "lonbin"]).mean()
    
    #count
    dnb = dfm.reset_index()[['obs', 'latbin', 'lonbin']].groupby(["latbin", "lonbin"]).count().obs.compute().to_xarray()
    dsms = dd.compute().to_xarray()
    #attrs
    for v in list(dsms.variables) :
        if v in ds :
            dsms[v].attrs = ds[v].attrs
    #merge
    dso = xr.merge([dsms, dnb.rename('nb_coloc_bin')])
    dso['drifter_sat_year']=l
    dso = dso.expand_dims('drifter_sat_year')
    dso = dso.set_coords('drifter_sat_year')
    # center lon, lat bins + reindex to have same for all
    lon_bins, lat_bins = np.arange(-180, 180, dl), np.arange(-90, 90, dl)
    dso = dso.reindex({'lonbin':lon_bins, 'latbin':lat_bins})
    dso['lonbin'] = dso['lonbin']+dl/2
    dso['latbin'] = dso['latbin']+dl/2

    return dso

def run_mslonlat(l):
    """main execution code"""
    dsm = xr.open_dataset(os.path.join(matchup_dir, f'matchup_{l}.zarr'))[var+['drogue_status', 'alti___distance', 'alti___time_difference']].dropna('obs').chunk({'obs':500})
    # add filtered acc and coriolis
    zarr_cutoff = os.path.join(zarr_dir+'_ok','cutoff_matchup',"cutoff_matchup_"+l+".zarr")
    zarr_cutoff1 = os.path.join(zarr_dir+'_ok','cutoff_matchup',"cutoff_matchup_"+l+"_2.zarr")
    if os.path.isdir(zarr_cutoff):
        dsm = xr.merge([dsm, xr.open_dataset(zarr_cutoff),xr.open_dataset(zarr_cutoff1)], join='inner').chunk({'obs':1000})
        
    dsm = dsm.where(dsm.alti___distance<=DL, drop=True)
    dsm = dsm.where(dsm.alti___time_difference<=DT, drop=True).drop(['alti___distance', 'alti___time_difference'])
    dsmd = dsm.where(dsm.drogue_status, drop=True).drop('drogue_status')
    dsmnd = dsm.where(np.logical_not(dsm.drogue_status), drop=True).drop('drogue_status')
    dsm = dsm.drop('drogue_status')
    
    # all
    zarr_ms = os.path.join(zarr_dir+'_ok',f'mslonlat/mslonlat_{int(DL//1000)}_{DT}_{dl}_{l}.zarr')
    zarr_mean = os.path.join(zarr_dir+'_ok',f'meanlonlat/meanlonlat_{int(DL//1000)}_{DT}_{dl}_{l}.zarr')
    if not os.path.isdir(zarr_ms) :
        bin_lonlat_ms(dsm,l, dl).to_zarr(zarr_ms,encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')
    if not os.path.isdir(zarr_mean) :
        bin_lonlat_mean(dsm,l, dl).to_zarr(zarr_mean,encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')
        print(f"{l} ms/mean storred")
        
    #drogued   
    if dsmd.dims['obs']!=0 : 
        zarrd_ms = os.path.join(zarr_dir+'_ok',f'mslonlat/mslonlat_{int(DL//1000)}_{DT}_{dl}_drogued_{l}.zarr')
        zarrd_mean = os.path.join(zarr_dir+'_ok',f'meanlonlat/meanlonlat_{int(DL//1000)}_{DT}_{dl}_drogued_{l}.zarr')
        if not os.path.isdir(zarrd_ms) :
            bin_lonlat_ms(dsmd,l, dl).to_zarr(zarrd_ms, encoding={'drifter_sat_year':{'dtype':'U32'}},mode='w')
        if not os.path.isdir(zarrd_mean) :
            bin_lonlat_mean(dsmd,l, dl).to_zarr(zarrd_mean, encoding={'drifter_sat_year':{'dtype':'U32'}},mode='w')
        print(f"drogued {l} ms/mean storred")
        
    #undrogued    
    if dsmnd.dims['obs']!=0 :
        zarrud_ms = os.path.join(zarr_dir+'_ok',f'mslonlat/mslonlat_{int(DL//1000)}_{DT}_{dl}_undrogued_{l}.zarr')
        zarrud_mean = os.path.join(zarr_dir+'_ok',f'meanlonlat/meanlonlat_{int(DL//1000)}_{DT}_{dl}_undrogued_{l}.zarr')
        if not os.path.isdir(zarrud_ms) :
            bin_lonlat_ms(dsmnd,l, dl).to_zarr(zarrud_ms,encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')
        if not os.path.isdir(zarrud_mean) :
            bin_lonlat_mean(dsmnd,l, dl).to_zarr(zarrud_mean,encoding={'drifter_sat_year':{'dtype':'U32'}}, mode='w')
        print(f"undrogued {l} ms/mean storred")

In [5]:
run_mslonlat(l)

drogued gps_Sentinel-3_A_2019 ms/mean storred
undrogued gps_Sentinel-3_A_2019 ms/mean storred


# Test 1 label

In [11]:
files = glob(os.path.join(zarr_dir+'_ok',f'mslonlat/mslonlat_{int(DL//1000)}_{DT}_{dl}_drogued_*.zarr'))
files = [f for f in files if 'gps' in f]

ds = xr.open_dataset(files[3])

In [14]:
for f in files :
    ds = xr.open_dataset(f)
    print(len(ds))

66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66
66


In [7]:
cluster.close()